In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
# import torchaudio
import requests
import matplotlib.pyplot as plt
# from pydub import AudioSegment
import os
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import train_test_split
from scipy.sparse import csr_matrix
from sklearn.utils import shuffle

# torchaudio.USE_SOUNDFILE_LEGACY_INTERFACE = False
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 
print(device)
%cd '/content/drive/MyDrive/RS20201'

cuda
/content/drive/.shortcut-targets-by-id/139/RS20201


In [2]:
with open('/content/drive/MyDrive/RS20201/data/train.txt', 'r') as f:
  train_data = [line.split('<fff>') for line in f.read().splitlines()]

with open('/content/drive/MyDrive/RS20201/data/test.txt', 'r') as f:
  test_data = [line.split('<fff>') for line in f.read().splitlines()]


user_ids = list(set([line[0] for line in train_data]))
song_ids = list(set([line[1] for line in train_data]))

num_users = len(user_ids)
num_songs = len(song_ids)
print('num users:', num_users)
print('num songs:', num_songs)
user_ids_to_idx = dict(zip(user_ids, range(num_users)))
song_ids_to_idx = dict(zip(song_ids, range(num_songs)))

Y = np.zeros((num_users, num_songs))
max_rate = 0
def sigmoid(x):
    return 1.0/(1.0+np.exp(-x))

for i in range(len(train_data)):
    u = user_ids_to_idx[train_data[i][0]]
    s = song_ids_to_idx[train_data[i][1]]
    r = sigmoid(int(train_data[i][2]))
    if r > max_rate:
        max_rate = r
    Y[u][s] = r
    
Y = csr_matrix(Y)
print('max rate:',max_rate)
print('num rating:',len(train_data))
print('density:', len(train_data)/(num_users*num_songs))

train_data = dict([((line[0], line[1]), sigmoid(int(line[2]))) for line in train_data])
test_data = dict([((line[0], line[1]), sigmoid(int(line[2]))) for line in test_data])


num users: 33963
num songs: 5527
max rate: 1.0
num rating: 845387
density: 0.004503602540554609


In [3]:
mask = (Y>0)*1.0
Y_copy = Y.copy()
mask_copy = mask.copy()
mu = Y.sum()/mask.sum()
print(mu)


0.821734201016593


In [4]:
class AutoRec(nn.Module):
    def __init__(self, num_songs, embedding_size=64, n_hidden=256):
        super().__init__()
        self.num_songs = num_songs
        self.embedding_size = embedding_size
        self.n_hidden = n_hidden
        self.encoder = nn.Sequential(
            nn.Dropout(p=0.7),
            nn.Linear(num_songs, n_hidden),
            nn.ReLU(),
            nn.Linear(n_hidden, embedding_size),
            nn.ReLU()
        )
        self.decoder = nn.Sequential(
            nn.Linear(embedding_size, n_hidden),
            nn.ReLU(),
            nn.Linear(n_hidden, num_songs),
        )

    def forward(self, inp):
        latent = self.encoder(inp)
        out = self.decoder(latent)
        return out

In [5]:
def criterion(r_pred, r_true):
    _mask = (r_true!=0)*1.0
    diff = r_pred - r_true
    loss = diff*diff*_mask
    # print(torch.sum(loss))
    # print(torch.sum(_mask))
    return torch.sum(loss)/torch.sum(_mask)

def train(model, Y, mask, num_epochs=10, batch_size=128, lr=0.001):
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
    num_batchs = num_users//batch_size + 1
    for epoch in range(num_epochs):
        total_loss = 0
        Y, mask = shuffle(Y, mask)
        for i in range(num_batchs):
            optimizer.zero_grad()
            top = (i+1)*batch_size if (i+1)*batch_size < num_users else num_users
            inp = Y[i*batch_size: top].toarray()
            m = mask[i*batch_size: top].toarray()
            inp = inp - mu*m
            inp = torch.tensor(inp).float().to(device)
            out = model(inp)
            loss = criterion(out, inp)
            total_loss += loss.item()
            loss.backward()
            optimizer.step()
        if epoch % 10 == 0:
            print(epoch, total_loss/num_batchs)


In [12]:
model = AutoRec(num_songs).to(device)
model.load_state_dict(torch.load('/content/drive/MyDrive/RS20201/model.pt'))

def eval(model):
    model.eval()
    mae = []
    ndcg = []
    for u_id, s_id in test_data:
        if u_id not in user_ids_to_idx or s_id not in song_ids_to_idx:
            continue
        u = user_ids_to_idx[u_id]
        s = song_ids_to_idx[s_id]
        inp = Y_copy[u].toarray()[0]
        with torch.no_grad():
            pred = model(torch.tensor(inp).float().to(device)).cpu().numpy() + mu
        pred_list = []
        for i, r in enumerate(pred):
            if (u_id, song_ids[i]) not in train_data:
                pred_list.append((r, i))
        pred_list = sorted(pred_list, key=lambda x:x[0], reverse=True)
        for rank, (r, i) in enumerate(pred_list):
            if s == i:
                mae.append(np.abs(r-test_data[(u_id, s_id)]))
                ndcg.append(np.log(2)/np.log(rank+2))
                print(mae[-1], ndcg[-1])
                break

        

    return np.mean(mae), np.mean(ndcg)

print(eval(model))

0.09491159425207518 0.09023520584937315
0.08480258689352416 0.08663235236389465
0.13269924412437428 0.0878782625678148
0.06966241107412718 0.08349889710252908
0.036677886808143545 0.10120316433752355
0.14800696860023488 0.08420249109334492
0.13932457390097608 0.1000423464158209
0.08312710032888793 0.08617307191923339
0.18869423161216725 0.08074600288044467
0.1247189830345764 0.11409180267477462
0.023836721742378164 0.11479993357805733
0.12047543034979247 0.10911185547224338
0.030634869897590566 0.10740431628408452
0.04326843198560326 0.09653766504984769
0.08045025573202513 0.08558164428563603
0.11944051490255736 0.10785784260696445
0.08376543697141259 0.08300266762417653
0.05035581098028563 0.08151122082507703
0.05267300115057372 0.08165240015237213
0.17838980360117784 0.08755387589674161
0.05335082517095946 0.0816841243931177
0.08598848090597533 0.086957687069808
0.15463343825265619 0.09730193980829183
0.13738303671546925 0.08641733286283182
0.22862147072781025 0.0809854428899419
0.20

KeyboardInterrupt: ignored